In [1]:
import pandas as pd
import json

In [2]:
df = pd.read_csv('data/RIA_before_2025.csv').dropna().reset_index(drop=True).reset_index()
df

,index,date,year,month,day,text
0,0,2020-01-01 00:32:53,2020,1,1,"Трамп вновь пригрозил последствиями Ирану, есл..."
1,1,2020-01-01 00:54:16,2020,1,1,Ким Чен Ын пообещал скоро представить новое ст...
2,2,2020-01-01 12:44:25,2020,1,1,"В России почти 6,5 миллиона человек встретили ..."
3,3,2020-01-01 12:55:30,2020,1,1,Столкновения произошли в Гонконге во время сог...
4,4,2020-01-01 15:20:14,2020,1,1,"Папа Римский Франциск извинился за то, что нак..."
...,...,...,...,...,...,...
168730,168730,2024-12-31 18:21:42,2024,12,31,Заявка на транзит российского газа через Украи...
168731,168731,2024-12-31 19:45:52,2024,12,31,\n\n▪️Российские войска нанесли групповой удар...
168732,168732,2024-12-31 21:02:19,2024,12,31,"❗️Путин заслушал доклады начальника Генштаба, ..."
168733,168733,2024-12-31 21:05:02,2024,12,31,Повторные выбросы нефтепродуктов зафиксированы...


In [3]:
# df = df.drop(columns=['multi_labels', 'hier_label']).copy()

In [4]:
# import json
# import re
# from transformers import AutoModelForCausalLM, AutoTokenizer
# import pandas as pd

# MODEL_NAME = "yandex/YandexGPT-5-Lite-8B-pretrain"

# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, legacy=False)
# model = AutoModelForCausalLM.from_pretrained(
#     MODEL_NAME,
#     device_map="cuda",
#     torch_dtype="auto",
# )

In [5]:
# def generate_categories(news_text):
#     prompt = f"""
# Ты — редактор новостного агрегатора.  Твоя задача - категоризировать новость. Предоставь JSON:

# - **multi_labels**: Список (list) до 3 основных тем (Политика, Экономика, Общество, Происшествия, Спорт, Культура, Технологии, Международные отношения, Региональные новости).
# - **hier_label**: Список (list) [тема, подтема]. `тема` - первая из `multi_labels`, `подтема` - её уточнение (например, "Политика", "Внешняя политика").

# Входные данные - словарь (dict) с ключами: 'text' (текст новости).

# Текст новости: {news_text}

# Ответ:
# """
#     model.config.pad_token_id = tokenizer.pad_token_id or tokenizer.eos_token_id

#     input_ids = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024).to("cuda")
#     outputs = model.generate(**input_ids, max_length=2048)
#     response = tokenizer.decode(outputs[0], skip_special_tokens=True)
#     print(response)

#     # Извлекаем JSON из текста
#     json_match = re.search(r"\{.*\}", response, re.DOTALL)
#     if json_match:
#         json_text = json_match.group(0)
#         try:
#             return json.loads(json_text)
#         except json.JSONDecodeError:
#             return {"multi_labels": [], "hier_label": []}  # Возвращаем пустую структуру при ошибке
#     else:
#         return {"multi_labels": [], "hier_label": []}


# def process_first_n_rows(df, n):
#     """Обрабатывает первые n строк DataFrame, добавляя категории."""

#     # Создаем копию первых n строк, чтобы избежать SettingWithCopyWarning
#     df_subset = df.head(n).copy()

#     # Применяем функцию generate_categories к подмножеству DataFrame
#     df_subset["categories"] = df_subset["text"].apply(generate_categories)
#     df_subset["multi_labels"] = df_subset["categories"].apply(lambda x: x["multi_labels"])
#     df_subset["hier_label"] = df_subset["categories"].apply(lambda x: x["hier_label"])
#     df_subset.drop(columns=["categories"], inplace=True)

#     return df_subset

# N = 10
# df_processed = process_first_n_rows(df, N)

# df = df.drop(index=df.index[:N])  #удаляем старые N строк, которые мы обработали
# df = pd.concat([df_processed, df]).reset_index(drop=True) #объединяем обработанные строки с остальными

# df

In [6]:
# import json
# import re
# from transformers import AutoModelForCausalLM, AutoTokenizer
# import pandas as pd

# # --- Модифицированный код для модели T-lite ---
# MODEL_NAME = "t-tech/T-lite-it-1.0"

# # Инициализация модели и токенизатора
# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# model = AutoModelForCausalLM.from_pretrained(
#     MODEL_NAME,
#     device_map="auto",
#     torch_dtype="auto",
# )

# def generate_categories(news_text):
#     # Формируем системный промпт и запрос пользователя
#     messages = [
#         {
#             "role": "system",
#             "content": """Ты — редактор новостного агрегатора в России. Твоя задача: категоризировать новости для российских читателей, создав две колонки: multi_labels и hier_label.
            
#             multi_labels: Список [,] из наиболее значимых категорий новости.
#             hier_label: Список [,] - Двухуровневая иерархия для первой категории.
            
#             Необходим ответ без пояснений строго в формате JSON."""
#         },
#         {
#             "role": "user", 
#             "content": f"Текст новости: {news_text}"
#         }
#     ]
    
#     # Применяем чатовый шаблон
#     text = tokenizer.apply_chat_template(
#         messages,
#         tokenize=False,
#         add_generation_prompt=True
#     )
    
#     # Токенизация и генерация
#     model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
#     generated_ids = model.generate(
#         **model_inputs,
#         max_new_tokens=128,
#         pad_token_id=tokenizer.eos_token_id
#     )
    
#     # Извлекаем сгенерированный текст
#     generated_ids = [
#         output_ids[len(input_ids):] 
#         for input_ids, output_ids 
#         in zip(model_inputs.input_ids, generated_ids)
#     ]
#     response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
#     print(response)
#     # Парсинг JSON
#     try:
#         json_match = re.search(r"\{.*\}", response, re.DOTALL)
#         return json.loads(json_match.group(0)) if json_match else {"multi_labels": [], "hier_label": []}
#     except json.JSONDecodeError:
#         return {"multi_labels": [], "hier_label": []}

# # Остальной код обработки DataFrame остается без изменений
# def process_first_n_rows(df, n):
#     df_subset = df.head(n).copy()
#     df_subset["categories"] = df_subset["text"].apply(generate_categories)
#     df_subset["multi_labels"] = df_subset["categories"].apply(lambda x: x["multi_labels"])
#     df_subset["hier_label"] = df_subset["categories"].apply(lambda x: x["hier_label"])
#     df_subset.drop(columns=["categories"], inplace=True)
#     return df_subset

# # Параметры обработки
# N = 10  # Сколько строк обрабатывать за раз

# # Основной цикл обработки
# df_processed = process_first_n_rows(df, N)
# print("Обработанные данные:")
# print(df_processed)

# # Обновление основного DataFrame (как в твоем оригинальном коде)
# df = df.drop(index=df.index[:N])
# df = pd.concat([df_processed, df]).reset_index(drop=True)
# df

In [7]:
pd.set_option('display.max_colwidth', None)

In [8]:
df.head(10)

,index,date,year,month,day,text
0,0,2020-01-01 00:32:53,2020,1,1,"Трамп вновь пригрозил последствиями Ирану, если кто-то из американцев в Ираке пострадает от нападений: «Иран понесет полную ответственность за гибель людей или ущерб, причиненный какому-либо из наших объектов. Они заплатят очень большую цену! Это не предупреждение, а угроза. Счастливого нового года!»\n\nРанее протестующие проникли за внешний периметр ограды посольства США в Багдаде. США обвинили в нападении власти Ирана, Тегеран это отрицает."
1,1,2020-01-01 00:54:16,2020,1,1,"Ким Чен Ын пообещал скоро представить новое стратегическое оружие КНДР и заявил, что будет продолжать разработку стратегического оружия, пока враждебная политика США не будет устранена"
2,2,2020-01-01 12:44:25,2020,1,1,"В России почти 6,5 миллиона человек встретили Новый год, участвуя в праздничных мероприятиях, сообщило МВД РФ. Нарушений правопорядка не допущено. В МВД добавили, что общественный порядок охраняли около 67,4 тысячи полицейских."
3,3,2020-01-01 12:55:30,2020,1,1,"Столкновения произошли в Гонконге во время согласованной акции, полиция применила слезоточивый газ для разгона протестующих, сообщает газета South China Morning Post."
4,4,2020-01-01 15:20:14,2020,1,1,"Папа Римский Франциск извинился за то, что накануне отшлепал по руке одну не в меру активную паломницу.\n\nВо вторник вечером женщина не на шутку рассердила папу Франциска, который вышел на площадь Святого Петра в Ватикане, чтобы полюбоваться установленными там под елью рождественскими яслями и поприветствовать верующих. Понтифик по обычаю пожимал присутствующим руки и уже собирался вернуться в папские покои, когда паломница неожиданно схватила его за правую руку и бесцеремонно потянула ее к себе.\n\n"
5,5,2020-01-01 15:45:51,2020,1,1,"Более 75,5 тысяч пассажиров встретили Новый год в поездах дальнего следования, сообщила ""Федеральная пассажирская компания"""
6,6,2020-01-01 17:12:14,2020,1,1,"В России в ночь на 1 января , сообщает «Столото». Теперь счастливчика ждут для оформления выигрыша. Пока известно лишь то, что билет был куплен в Москве."
7,7,2020-01-01 19:38:39,2020,1,1,"Украинские силовики заявили, что в новом году , ранен военный.\n\nНесмотря на то что ""нормандские лидеры"" призывали до конца 2019 года обеспечить прекращение огня."
8,8,2020-01-02 00:51:53,2020,1,2,"Экс-глава Национальной баскетбольной ассоциации Дэвид Стерн скончался в среду в возрасте 77 лет.\n\nСпортивный менеджер встал у руля NBA в 1984 году и возглавлял ее три десятка лет, сделав одной из самых богатых спортивных лиг в мире - третьей в Северной Америке после NFL (американский футбол) и MLB (бейсбол). В декабре Стерн был госпитализирован из-за кровоизлияния в мозг."
9,9,2020-01-02 03:52:11,2020,1,2,"МЧС хочет из зон чрезвычайных ситуаций при соответствующем решении властей.\n\nВ законопроекте, в частности, предусмотрены ""возможность принудительной эвакуации населения из зон ЧС"" и ""введение ответственности за несвоевременное представление информации о ЧС""."


# Gemini

In [9]:
# import pandas as pd
# import json
# import re
# import time
# import math
# import google.generativeai as genai
# from IPython.display import display, HTML # Для display

# # --- НАСТРОЙКИ ---
# API_KEY_FILE = "config.py"
# MODEL_NAME = "gemini-2.5-flash-preview-04-17"

# TARGET_TOKENS_PER_BATCH = 15000 # Начнем с этого, как в успешном тесте
# DAILY_REQUEST_LIMIT = 480
# REQUEST_DELAY_SECONDS = 2.23 # Немного увеличим на всякий случай для preview
# SAMPLE_SIZE_FOR_TESTING = 1000 # Количество строк для тестового запуска

# # --- КОНЕЦ НАСТРОЕК ---

# # Загрузка API ключа
# try:
#     from importlib import import_module
#     config_module = import_module(API_KEY_FILE.replace('.py', ''))
#     API_KEY = config_module.API_KEY
# except (ImportError, AttributeError):
#     print(f"Ошибка: Не удалось загрузить API_KEY из {API_KEY_FILE}.")
#     exit()

# # Конфигурация Google GenAI
# try:
#     genai.configure(api_key=API_KEY)
# except Exception as e:
#     print(f"Ошибка конфигурации Google GenAI: {e}")
#     exit()

# generation_config = {
#   "temperature": 0.2, # Вернул к более низкому для точности
#   "max_output_tokens": 65536,
#   "response_mime_type": "application/json",
# }

# safety_settings = [
#     {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_NONE"},
#     {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_NONE"},
#     {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_NONE"},
#     {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_NONE"},
# ]

# gemini_model = genai.GenerativeModel(
#     model_name=MODEL_NAME,
#     generation_config=generation_config,
#     safety_settings=safety_settings
# )

# PROMPT_INSTRUCTION_TEMPLATE = """
# Ты — редактор новостного агрегатора. Твоя задача - категоризировать список новостей.
# Для каждой новости из списка предоставь JSON-объект.
# Каждый такой объект должен содержать:
# - "id": оригинальный идентификатор новости (целое число), который был предоставлен во входных данных.
# - "multi_labels": Список (list) до 3 основных тем новости. Допустимые темы: Политика, Экономика, Общество, Происшествия, Спорт, Культура, Технологии, Международные отношения, Региональные новости.
# - "hier_label": Список (list) из двух элементов: [тема, подтема]. `тема` - это первая тема из `multi_labels`. `подтема` - это уточнение для этой основной темы (например, если тема "Политика", подтема может быть "Внешняя политика" или "Выборы").
# Входные данные - это JSON-массив объектов, где каждый объект имеет "id" и "text".
# Твой ответ ДОЛЖЕН БЫТЬ JSON-массивом, где каждый элемент массива - это JSON-объект с полями "id", "multi_labels", "hier_label" для соответствующей входной новости.
# Убедись, что идентификаторы 'id' в твоем ответе точно соответствуют идентификаторам 'id' из входного списка новостей.
# Не включай в ответ ничего, кроме этого JSON-массива.
# Новости для обработки:
# {news_json_payload}
# Ответ (только JSON-массив):
# """

# try:
#     base_prompt_for_counting = PROMPT_INSTRUCTION_TEMPLATE.format(news_json_payload="[]")
#     TOKENS_FOR_BASE_PROMPT = gemini_model.count_tokens(base_prompt_for_counting).total_tokens
#     print(f"Токены, занимаемые базовой инструкцией промпта: {TOKENS_FOR_BASE_PROMPT}")
# except Exception as e:
#     print(f"Не удалось посчитать токены для базового промпта: {e}. Используем примерное значение 350.")
#     TOKENS_FOR_BASE_PROMPT = 350 # Запасное значение

# def generate_categories_batch_gemini(news_batch_items):
#     news_input_for_prompt = [{"id": item["id"], "text": item["text"]} for item in news_batch_items]
#     news_input_json_string = json.dumps(news_input_for_prompt, ensure_ascii=False, indent=2)
#     full_prompt = PROMPT_INSTRUCTION_TEMPLATE.format(news_json_payload=news_input_json_string)
    
#     # Подготавливаем "пустой" результат на случай ошибок, но с правильными ID
#     empty_results_with_ids = [{"id": item["id"], "multi_labels": [], "hier_label": []} for item in news_batch_items]

#     try:
#         response = gemini_model.generate_content(full_prompt)
        
#         # Отладочный вывод по ответу
#         print(f"  DEBUG_API: Запрос для IDs с {news_batch_items[0]['id']} по {news_batch_items[-1]['id']} ({len(news_batch_items)} шт.)")
#         if not response.parts:
#             print(f"  ПРЕДУПРЕЖДЕНИЕ_API: Пустой ответ (response.parts пуст). Возможно, заблокировано.")
#             if hasattr(response, 'prompt_feedback') and response.prompt_feedback:
#                  print(f"    Причина блокировки: {response.prompt_feedback}")
#             return empty_results_with_ids # Возвращаем пустые результаты с ID

#         response_text = response.text
#         print(f"  DEBUG_API: Длина response_text: {len(response_text)} символов.")
#         if len(response_text) < 50: # Если ответ очень короткий, это подозрительно
#              print(f"    Короткий response_text: '{response_text}'")
        
#         if not response_text.strip(): # Если текст пустой или только пробелы
#             print(f"  ПРЕДУПРЕЖДЕНИЕ_API: response_text пуст или состоит из пробелов.")
#             return empty_results_with_ids

#         try:
#             parsed_json_list = json.loads(response_text)
#             if not isinstance(parsed_json_list, list):
#                 print(f"  ОШИБКА_JSON: Ответ API не является JSON-массивом. Тип: {type(parsed_json_list)}. Ответ (начало): {response_text[:100]}...")
#                 return empty_results_with_ids
            
#             # Валидация и сопоставление результатов
#             # Создаем словарь из полученных результатов для быстрого доступа по id
#             results_map_from_api = {item.get("id"): item for item in parsed_json_list if isinstance(item, dict)}
            
#             # Формируем итоговый список для этого батча, гарантируя наличие всех запрошенных ID
#             batch_final_results = []
#             for requested_item in news_batch_items:
#                 req_id = requested_item["id"]
#                 if req_id in results_map_from_api:
#                     api_item = results_map_from_api[req_id]
#                     # Проверяем наличие и тип ключей
#                     if isinstance(api_item.get("multi_labels"), list) and isinstance(api_item.get("hier_label"), list):
#                         batch_final_results.append(api_item)
#                     else:
#                         print(f"  ПРЕДУПРЕЖДЕНИЕ_JSON: Некорректная структура категорий для id {req_id} в ответе API. Данные: {api_item}")
#                         batch_final_results.append({"id": req_id, "multi_labels": [], "hier_label": []})
#                 else:
#                     # Этого не должно происходить, если модель следует инструкциям, но лучше предусмотреть
#                     print(f"  ПРЕДУПРЕЖДЕНИЕ_JSON: ID {req_id} из запроса не найден в ответе API.")
#                     batch_final_results.append({"id": req_id, "multi_labels": [], "hier_label": []})
#             return batch_final_results

#         except json.JSONDecodeError as e:
#             print(f"  ОШИБКА_JSON: Ошибка декодирования JSON: {e}\n    Текст ответа (начало): {response_text[:300]}...")
#             return empty_results_with_ids

#     except Exception as e:
#         print(f"  КРИТИЧЕСКАЯ_ОШИБКА_API: При вызове API Gemini: {e}")
#         return empty_results_with_ids


# def process_dataframe_in_token_batches(df_input,
#                                        target_tokens_total=TARGET_TOKENS_PER_BATCH,
#                                        base_prompt_tokens=TOKENS_FOR_BASE_PROMPT,
#                                        daily_request_limit=DAILY_REQUEST_LIMIT,
#                                        request_delay=REQUEST_DELAY_SECONDS):
#     df = df_input.copy()
#     all_processed_results_flat = [] # Будет "плоским" списком словарей
#     current_batch_items = []
#     current_batch_estimated_tokens = base_prompt_tokens
#     requests_count = 0

#     df_results = df.copy() # Копия для модификации
#     df_results["multi_labels"] = pd.Series([[] for _ in range(len(df_results))], index=df_results.index, dtype=object)
#     df_results["hier_label"] = pd.Series([[] for _ in range(len(df_results))], index=df_results.index, dtype=object)


#     print("Предварительный подсчет токенов для каждой новости...")
#     news_with_tokens_info = []
#     JSON_OVERHEAD_PER_ITEM_APPROX = 20 # Обновлено на основе ваших логов
    
#     start_time_counting = time.time()
#     for i, (idx, row) in enumerate(df.iterrows()):
#         if (i + 1) % 250 == 0: print(f"  Подсчет токенов: обработано {i + 1}/{len(df)}...")
        
#         news_text = str(row["text"])
#         tokens_for_item = (len(news_text) // 4) + JSON_OVERHEAD_PER_ITEM_APPROX # Fallback: грубая оценка
#         try:
#             tokens_for_text_only = gemini_model.count_tokens(news_text).total_tokens
#             tokens_for_item = tokens_for_text_only + JSON_OVERHEAD_PER_ITEM_APPROX
#         except Exception as e:
#             print(f"    Предупреждение: Ошибка подсчета токенов для id {idx} ('{str(e)[:50]}...'). Используется грубая оценка.")
#         news_with_tokens_info.append({"id": idx, "text": news_text, "tokens": tokens_for_item})
#     print(f"Предварительный подсчет токенов завершен. Заняло: {time.time() - start_time_counting:.2f}s")

#     for news_item in news_with_tokens_info:
#         if requests_count >= daily_request_limit:
#             print(f"Достигнут дневной лимит запросов ({requests_count}). Обработка остановлена.")
#             break

#         item_tokens = news_item["tokens"]
#         # Если текущий батч не пуст И добавление нового элемента превысит лимит
#         if current_batch_items and (current_batch_estimated_tokens + item_tokens > target_tokens_total):
#             print(f"Отправка батча ({len(current_batch_items)} новостей, ~{current_batch_estimated_tokens} токенов). Запрос #{requests_count + 1}")
#             batch_results_list = generate_categories_batch_gemini(current_batch_items) # должен вернуть список словарей
#             requests_count += 1
#             if isinstance(batch_results_list, list): # Убедимся, что это список
#                  all_processed_results_flat.extend(batch_results_list)
            
#             current_batch_items = []
#             current_batch_estimated_tokens = base_prompt_tokens
#             if requests_count < daily_request_limit and request_delay > 0: time.sleep(request_delay)

#         # Если новость сама по себе (с базовым промптом) слишком велика
#         if base_prompt_tokens + item_tokens > target_tokens_total:
#             print(f"ПРЕДУПРЕЖДЕНИЕ: Новость id {news_item['id']} ({item_tokens} токенов) "
#                   f"слишком велика даже для одиночной отправки с базовым промптом (лимит батча: {target_tokens_total}). "
#                   "Новость будет пропущена.")
#             # Добавляем "пустой" результат для этой новости, чтобы сохранить ID в all_processed_results_flat
#             all_processed_results_flat.append({"id": news_item["id"], "multi_labels": [], "hier_label": []})
#             continue # Переходим к следующей новости

#         # Добавляем текущую новость в батч
#         current_batch_items.append(news_item)
#         current_batch_estimated_tokens += item_tokens

#     # Отправляем последний оставшийся батч
#     if current_batch_items and requests_count < daily_request_limit:
#         print(f"Отправка последнего батча ({len(current_batch_items)} новостей, ~{current_batch_estimated_tokens} токенов). Запрос #{requests_count + 1}")
#         batch_results_list = generate_categories_batch_gemini(current_batch_items)
#         requests_count += 1
#         if isinstance(batch_results_list, list):
#             all_processed_results_flat.extend(batch_results_list)

#     print(f"Всего сделано запросов к API: {requests_count}")

#     # Мерджим результаты в DataFrame
#     if all_processed_results_flat:
#         results_df = pd.DataFrame(all_processed_results_flat)
        
#         print(f"DEBUG_MERGE: ---- Анализ перед мержем ----")
#         print(f"DEBUG_MERGE: Всего элементов в results_df (из all_processed_results_flat): {len(results_df)}")
#         if not results_df.empty:
#             display(HTML("<b>results_df.head() перед мержем:</b>"))
#             display(results_df.head())
#             print(f"DEBUG_MERGE: dtypes results_df: \n{results_df.dtypes}")
#             print(f"DEBUG_MERGE: dtypes df_results.index: {df_results.index.dtype}")
        
#         if not results_df.empty and 'id' in results_df.columns:
#             # Приведение типов ID для корректного мержа
#             original_id_type = df_results.index.dtype
#             results_df['id_original_type'] = results_df['id'] # Сохраним для отладки

#             try:
#                 if pd.api.types.is_numeric_dtype(original_id_type):
#                     results_df['id'] = pd.to_numeric(results_df['id'], errors='coerce')
#                     results_df.dropna(subset=['id'], inplace=True)
#                     if not results_df.empty:
#                        results_df['id'] = results_df['id'].astype(original_id_type)
#                 elif original_id_type == 'object':
#                      results_df['id'] = results_df['id'].astype(str)
#                 # Если другие типы индекса - можно добавить обработку
#             except Exception as e:
#                 print(f"ПРЕДУПРЕЖДЕНИЕ_MERGE: Ошибка приведения типов ID: {e}. Мерж может быть неполным.")

#             # Мерж с использованием индекса
#             # Сначала установим индекс в results_df для эффективного обновления
#             if not results_df.empty and 'id' in results_df.columns and results_df['id'].notna().all():
#                 try:
#                     results_df = results_df.set_index('id')
#                     # Обновляем df_results используя .loc и данные из results_df
#                     # Это более pandas-way, чем итерация
#                     updatable_idx = df_results.index.intersection(results_df.index)
                    
#                     if not updatable_idx.empty:
#                         print(f"DEBUG_MERGE: Найдено {len(updatable_idx)} совпадающих ID для обновления.")
#                         # Прямое присвоение списков может вызывать проблемы, если не использовать .loc или .at
#                         # Более безопасный способ - через apply или поэлементное присвоение,
#                         # но для списков в ячейках object это должно работать.
#                         # Однако, чтобы избежать ValueErrors, лучше итерировать по совпадающим индексам.
#                         for idx_to_update in updatable_idx:
#                             res_row = results_df.loc[idx_to_update]
#                             df_results.at[idx_to_update, "multi_labels"] = res_row.get("multi_labels", [])
#                             df_results.at[idx_to_update, "hier_label"] = res_row.get("hier_label", [])
#                     else:
#                         print("DEBUG_MERGE: Не найдено совпадающих ID для обновления.")
#                 except Exception as e:
#                     print(f"ОШИБКА_MERGE: при установке индекса или обновлении df_results: {e}")
#             else:
#                 print("ПРЕДУПРЕЖДЕНИЕ_MERGE: results_df пуст после приведения типов ID или содержит NaN в ID.")
#         else:
#             print("ПРЕДУПРЕЖДЕНИЕ_MERGE: DataFrame с результатами (results_df) пуст или не содержит колонку 'id'.")
#     else:
#         print("Не было получено результатов от API (all_processed_results_flat пуст).")
#     return df_results

# df_main = df.reset_index(drop=True).copy()


# # --- Подготовка и запуск обработки ---
# if len(df_main) > SAMPLE_SIZE_FOR_TESTING:
#     df_to_process = df_main.head(SAMPLE_SIZE_FOR_TESTING).copy()
#     print(f"Используется выборка из {SAMPLE_SIZE_FOR_TESTING} строк для обработки.")
# else:
#     df_to_process = df_main.copy()
#     print(f"Используется полный DataFrame ({len(df_main)} строк) для обработки.")

# CURRENT_RUN_TARGET_TOKENS = TARGET_TOKENS_PER_BATCH

# print(f"\n--- Запуск обработки с целевым лимитом токенов на батч: {CURRENT_RUN_TARGET_TOKENS} ---")
# start_processing_time = time.time()
# df_processed = process_dataframe_in_token_batches(
#     df_to_process,
#     target_tokens_total=CURRENT_RUN_TARGET_TOKENS,
#     base_prompt_tokens=TOKENS_FOR_BASE_PROMPT,
#     daily_request_limit=DAILY_REQUEST_LIMIT,
#     request_delay=REQUEST_DELAY_SECONDS
# )
# end_processing_time = time.time()
# print(f"Общее время обработки: {end_processing_time - start_processing_time:.2f} секунд")

# print("\n--- Обработанный DataFrame (первые 20 строк) ---")
# display(df_processed.head(20))

# filled_rows = df_processed[df_processed['multi_labels'].apply(lambda x: isinstance(x, list) and len(x) > 0)].shape[0]
# print(f"\nКоличество строк с заполненными категориями: {filled_rows} из {len(df_processed)}")

# # Дополнительная проверка для строк, где метки остались пустыми
# empty_label_rows = df_processed[df_processed['multi_labels'].apply(lambda x: isinstance(x, list) and len(x) == 0)]
# if not empty_label_rows.empty:
#     print(f"\nСтроки, где метки остались пустыми ({len(empty_label_rows)} шт.):")
#     display(empty_label_rows.head())

In [10]:
# df_processed.to_csv('1k.csv', index=False)

In [11]:
# df_processed.tail(50)

In [12]:
# from google import genai

# import config  # Импортируем файл конфигурации

# client = genai.Client(api_key=config.API_KEY)  # Используем API ключ из конфигурации

# response = client.models.generate_content(
#     model="gemini-2.0-flash",
#     contents="Здарова бандит",
# )

# print(response.text)

In [13]:
import pandas as pd
import json
import re # Если используется где-то еще
import time
import math
import google.generativeai as genai # Для подсчета TOKENS_FOR_BASE_PROMPT
from IPython.display import display, HTML
import numpy as np
import multiprocessing
import os
import warnings

# Импортируем функции из нового файла
from gemini_workers import _process_chunk_with_token_batches_for_worker, PROMPT_INSTRUCTION_TEMPLATE # PROMPT_INSTRUCTION_TEMPLATE нужен для подсчета TOKENS_FOR_BASE_PROMPT
warnings.filterwarnings("ignore", category=FutureWarning, module="numpy._core.fromnumeric", message=".*swapaxes.*")

# --- НАСТРОЙКИ ---
API_KEY_FILE = "config.py"
DEFAULT_MODEL_NAME = "gemini-2.5-flash-preview-04-17"
TARGET_TOKENS_PER_BATCH = 10000
DAILY_REQUEST_LIMIT_PER_KEY = 450
REQUEST_DELAY_SECONDS = 2.23
ROWS_PER_CHUNK = 2000
PARTIAL_RESULTS_FILE = "processed_news_partial.pkl"
SAVE_PROGRESS_EVERY_N_CHUNKS = 1
# --- КОНЕЦ НАСТРОЕК ---

# Загрузка API ключей (остается здесь)
try:
    from importlib import import_module
    config_module = import_module(API_KEY_FILE.replace('.py', ''))
    API_KEYS = config_module.API_KEYS
    if not API_KEYS or not isinstance(API_KEYS, list) or not all(isinstance(key, str) for key in API_KEYS):
        raise ValueError("API_KEYS должен быть непустым списком строк.")
    print(f"Загружено {len(API_KEYS)} API ключей.")
except Exception as e:
    print(f"Ошибка: Не удалось загрузить API_KEYS из {API_KEY_FILE}. {e}")
    exit()

# Инициализация модели ТОЛЬКО для подсчета TOKENS_FOR_BASE_PROMPT в основном процессе
# Воркеры будут инициализировать свои модели
try:
    # Используем первый ключ для этой одноразовой операции
    genai.configure(api_key=API_KEYS[0])
    temp_model_for_counting = genai.GenerativeModel(model_name=DEFAULT_MODEL_NAME) # Используем модель по умолчанию
    base_prompt_for_counting_main = PROMPT_INSTRUCTION_TEMPLATE.format(news_json_payload="[]")
    TOKENS_FOR_BASE_PROMPT_MAIN = temp_model_for_counting.count_tokens(base_prompt_for_counting_main).total_tokens
    print(f"Токены, занимаемые базовой инструкцией промпта (посчитано в основном потоке): {TOKENS_FOR_BASE_PROMPT_MAIN}")
    del temp_model_for_counting # Удаляем временную модель
except Exception as e:
    print(f"Не удалось посчитать токены для базового промпта в основном потоке: {e}. Используем примерное значение 350.")
    TOKENS_FOR_BASE_PROMPT_MAIN = 350


# --- Основная логика (остается здесь, но вызывает импортированную функцию) ---
if __name__ == '__main__':
    multiprocessing.freeze_support()

    # 1. Загружаем CSV С КОЛОНКОЙ 'date' и 'text' ДЛЯ ФИЛЬТРАЦИИ
    df_temp_for_filtering = pd.read_csv('data/RIA_before_2025.csv', usecols=['date', 'text'])
    print(f"Загружен DataFrame из 'data/RIA_before_2025.csv', строк до фильтрации: {len(df_temp_for_filtering)}")
    # 2. Преобразуем колонку 'date' в datetime объекты
    df_temp_for_filtering['date'] = pd.to_datetime(df_temp_for_filtering['date'])
    # 3. Удаляем строки, где дата не смогла распарситься или где текст пуст
    df_temp_for_filtering.dropna(subset=['date', 'text'], inplace=True)
    print(f"Строк после удаления NaT в дате и пустых текстов: {len(df_temp_for_filtering)}")
    # 4. Фильтруем по 2024 году
    df_filtered_by_year = df_temp_for_filtering[df_temp_for_filtering['date'].dt.year == 2024].copy()
    print(f"Строк после фильтрации по 2024 году: {len(df_filtered_by_year)}")
    # 5. ОСТАВЛЯЕМ ТОЛЬКО КОЛОНКУ 'text' для df_main_full
    if not df_filtered_by_year.empty:
        df_main_full = df_filtered_by_year[['text']].copy() # Выбираем только колонку text
    else:
        df_main_full = pd.DataFrame(columns=['text']) # Создаем пустой DataFrame со структурой, если после фильтра нет данных
    # 6. Сбрасываем индекс и назначаем имя 'id'
    df_main_full.reset_index(drop=True, inplace=True)
    df_main_full.index.name = 'id'
    print(f"Загружен DataFrame из 'data/RIA_before_2025.csv', строк: {len(df_main_full)}")

    # Загрузка ранее обработанных результатов
    if os.path.exists(PARTIAL_RESULTS_FILE):
        print(f"Найден файл с частичными результатами: {PARTIAL_RESULTS_FILE}. Загрузка...")
        df_overall_results = pd.read_pickle(PARTIAL_RESULTS_FILE)
        for col in ["multi_labels", "hier_label"]:
            if col not in df_overall_results.columns:
                df_overall_results[col] = pd.Series([[] for _ in range(len(df_overall_results))], index=df_overall_results.index, dtype=object)
            else:
                df_overall_results[col] = df_overall_results[col].astype(object).apply(lambda x: x if isinstance(x, list) else [])
        processed_ids = set(df_overall_results.index)
        print(f"Загружено {len(processed_ids)} ранее обработанных ID.")
        df_to_process_now = df_main_full[~df_main_full.index.isin(processed_ids)].copy()
    else:
        print(f"Файл с частичными результатами {PARTIAL_RESULTS_FILE} не найден. Начинаем с нуля.")
        df_to_process_now = df_main_full.copy()
        df_overall_results = df_main_full.copy()
        df_overall_results["multi_labels"] = pd.Series([[] for _ in range(len(df_overall_results))], index=df_overall_results.index, dtype=object)
        df_overall_results["hier_label"] = pd.Series([[] for _ in range(len(df_overall_results))], index=df_overall_results.index, dtype=object)

    if df_to_process_now.empty:
        print("Все новости уже обработаны согласно файлу частичных результатов.")
        display(HTML("<b>Итоговый обработанный DataFrame (из файла):</b>"))
        if not df_overall_results.empty: display(df_overall_results.head(20))
    else:
        print(f"Всего новостей для обработки в этой сессии: {len(df_to_process_now)}")
        num_chunks = math.ceil(len(df_to_process_now) / ROWS_PER_CHUNK)
        if num_chunks == 0 and len(df_to_process_now) > 0 : num_chunks = 1 # Если строк меньше чем ROWS_PER_CHUNK
        
        if num_chunks > 0:
            chunks_dfs = np.array_split(df_to_process_now, num_chunks)
            print(f"Данные разделены на {len(chunks_dfs)} чанков по ~{ROWS_PER_CHUNK} строк.")
        else:
            chunks_dfs = []
            print("Нет данных для разделения на чанки (df_to_process_now пуст, но не должен быть здесь).")


        worker_args = []
        for i, chunk_df_arg in enumerate(chunks_dfs):
            api_key_for_chunk = API_KEYS[i % len(API_KEYS)]
            worker_args.append((
                chunk_df_arg, api_key_for_chunk, i + 1, DEFAULT_MODEL_NAME,
                TARGET_TOKENS_PER_BATCH, DAILY_REQUEST_LIMIT_PER_KEY, REQUEST_DELAY_SECONDS
            ))

        start_total_processing_time = time.time()
        pool_size = min(len(API_KEYS), len(chunks_dfs) if chunks_dfs else 1) # Не больше воркеров, чем чанков
        if pool_size > 0:
             print(f"Запуск обработки с пулом из {pool_size} воркеров.")
        else:
            print("Нет чанков для обработки.")


        for i in range(0, len(worker_args), pool_size):
            current_super_batch_args = worker_args[i : i + pool_size]
            if not current_super_batch_args: break
            print(f"\n--- Обработка супер-батча чанков с {i + 1} по {i + len(current_super_batch_args)} (всего {len(chunks_dfs)}) ---")
            
            with multiprocessing.Pool(processes=pool_size) as pool:
                try:
                    # Используем импортированную функцию
                    list_of_processed_chunk_dfs = pool.starmap(_process_chunk_with_token_batches_for_worker, current_super_batch_args)
                    
                    # Обновление df_overall_results новыми данными из этого супер-батча
                    for processed_chunk_df in list_of_processed_chunk_dfs:
                        if processed_chunk_df is not None and not processed_chunk_df.empty:
                            common_indices = df_overall_results.index.intersection(processed_chunk_df.index)
                            for col_to_update in ["multi_labels", "hier_label"]:
                                df_overall_results.loc[common_indices, col_to_update] = processed_chunk_df.loc[common_indices, col_to_update]
                except Exception as e_pool:
                    print(f"ОШИБКА ПУЛА ВОРКЕРОВ: {e_pool}")

            # Сохранение после каждого супер-батча
            try:
                df_overall_results.to_pickle(PARTIAL_RESULTS_FILE)
                print(f"Прогресс сохранен в {PARTIAL_RESULTS_FILE} (обработано до чанка #{i + len(current_super_batch_args)})")
            except Exception as e_save:
                print(f"Ошибка сохранения файла {PARTIAL_RESULTS_FILE}: {e_save}")

        end_total_processing_time = time.time()
        print(f"\nОбщее время всей обработки: {end_total_processing_time - start_total_processing_time:.2f} секунд")
        display(HTML("<b>Итоговый обработанный DataFrame:</b>"))
        if not df_overall_results.empty: display(df_overall_results.head(20))
        filled_rows_total = df_overall_results[df_overall_results['multi_labels'].apply(lambda x: isinstance(x, list) and len(x) > 0)].shape[0]
        print(f"\nИтого: количество строк с заполненными категориями: {filled_rows_total} из {len(df_overall_results)}")
        empty_label_rows_total = df_overall_results[df_overall_results['multi_labels'].apply(lambda x: isinstance(x, list) and len(x) == 0)]
        if not empty_label_rows_total.empty:
            print(f"\nСтроки, где метки остались пустыми ({len(empty_label_rows_total)} шт.):")
            if not empty_label_rows_total.empty: display(empty_label_rows_total.head())

Загружено 6 API ключей.


Токены, занимаемые базовой инструкцией промпта (посчитано в основном потоке): 346
Загружен DataFrame из 'data/RIA_before_2025.csv', строк до фильтрации: 186879
Строк после удаления NaT в дате и пустых текстов: 168735
Строк после фильтрации по 2024 году: 32963
Загружен DataFrame из 'data/RIA_before_2025.csv', строк: 32963
Файл с частичными результатами processed_news_partial.pkl не найден. Начинаем с нуля.
Всего новостей для обработки в этой сессии: 32963
Данные разделены на 17 чанков по ~2000 строк.
Запуск обработки с пулом из 6 воркеров.

--- Обработка супер-батча чанков с 1 по 6 (всего 17) ---
[Воркер 1, Ключ ..1uew] Начинает обработку чанка из 1939 строк.
[Воркер 2, Ключ ..pVWw] Начинает обработку чанка из 1939 строк.
[Воркер 3, Ключ ..fSgk] Начинает обработку чанка из 1939 строк.
[Воркер 4, Ключ ..NMZQ] Начинает обработку чанка из 1939 строк.
[Воркер 5, Ключ ..dhKY] Начинает обработку чанка из 1939 строк.
[Воркер 6, Ключ ..8osk] Начинает обработку чанка из 1939 строк.
  [Воркер 3, 

,text,multi_labels,hier_label
id,,,
0,Новогоднее обращение Владимира Путина,"[Политика, Общество]","[Политика, Президент]"
1,"Украинские войска через 10 минут после наступления нового года выпустили 15 ракет из РСЗО по Донецку, сообщили в представительстве ДНР в СЦКК","[Происшествия, Международные отношения, Региональные новости]","[Происшествия, Военные действия]"
2,❗️Пушилин сообщил о трех погибших в результате обстрела Донецка в новогоднюю ночь,"[Происшествия, Международные отношения, Региональные новости]","[Происшествия, Жертвы конфликта]"
3,"Воздушная тревога объявлена в Киеве и Киевской области, а также в Винницкой, Черкасской, Николаевской, Одесской областях и на подконтрольной Киеву территории Херсонской области","[Происшествия, Международные отношения, Региональные новости]","[Происшествия, Военные действия]"
4,"В Улан-Удэ из-за запущенного во дворе фейерверка загорелся балкон на 9-м этаже 16-этажного дома.\n\nПострадавших нет, спасатели потушили пожар на 2 квадратных метрах (видео от МЧС РФ).","[Происшествия, Региональные новости]","[Происшествия, Пожары]"
5,"Ким Чен Ын на новогодней встрече с командованием армии заявил, что если США и Южная Корея выберут путь конфронтации и вызовут конфликт,","[Международные отношения, Политика]","[Международные отношения, Военная политика]"
6,"Вулкан Ключевской на Камчатке в ходе нового извержения выбросил столб пепла до 6 км, шлейф пепла протянулся более чем на 100 км.\n\nПредыдущее извержение длилось с 23 июня по 1 ноября и завершилось крупным выбросом пепла на высоту до 14 км. Частицы пепла тогда выпали в Скандинавии.\n\nФото Камчатской группы реагирования на вулканические извержения Института вулканологии и сейсмологии ДВО РАН.","[Происшествия, Региональные новости]","[Происшествия, Стихийные бедствия]"
7,"Объект припортовой инфраструктуры поврежден в Одессе из-за взрывов, там произошел пожар, сообщили украинские военные","[Происшествия, Международные отношения, Региональные новости]","[Происшествия, Военные действия]"
8,"Музей украинскому националисту Шухевичу уничтожен во Львове из-за взрывов, сообщил мэр города","[Происшествия, Международные отношения, Культура]","[Происшествия, Военные действия]"



Итого: количество строк с заполненными категориями: 32902 из 32963

Строки, где метки остались пустыми (61 шт.):


,text,multi_labels,hier_label
id,,,
6970,Трехдневный траур объявлен в Кемеровской области в связи с трагедией в Красногорске,[],[]
6971,"Посетительницы концерта в ""Крокусе"" рассказали РИА Новости, что террористы стреляли людям в спину, когда те убегали из зала, и что убегавшие от преступников не могли сразу выбраться из здания из-за заблокированных дверей.\n\n""В зале они кричали: ""Лежать! Лежать!"", а в холле раздавались крики ""Стоять!"" и выстрелы за спиной, стреляли в спины"", - сказала РИА Новости одна из очевидец.\n\n""Кричали. Стреляли в спины и кричали: ""Всем лежать, всем не двигаться"". Не знаю, как нам удалось выбраться (из зала), мы сидели не очень далеко от двери, благо дверь была открыта"", - рассказала её подруга.\n\nЕще один очевидец прислал РИА Новости видео эвакуации: выбраться людям из здания, после того, как они покинули зал, удалось не сразу, потому что один из выходов был закрыт.",[],[]
6972,"❗️Один из задержанных по подозрению в теракте в ""Крокусе"" признался, что прилетел из Турции 4 марта. Видео допроса опубликовала Симоньян.\n\nТакже он рассказал, что ему предлагали за убийство людей.\n\nНа опубликованных кадрах мужчина называет свое имя, которое звучит схоже с Фариддун Шамсутдин, а также дату рождения - 17 сентября 1998.",[],[]
6973,"Глава Евросовета Шарль Мишель осудил теракт в ""Крокусе"", выразил соболезнования",[],[]
6974,❗️Патриарх Кирилл после теракта в Подмосковье поручил усилить безопасность храмов Московской епархии,[],[]


In [93]:
df_filtered = df_overall_results[df_overall_results['hier_label'].apply(lambda x: isinstance(x, list) and len(x) > 0)].copy()
display(df_filtered)
df_filtered.to_csv('2024_y.csv', index=False)


,text,multi_labels,hier_label
id,,,
0,Новогоднее обращение Владимира Путина,"[Политика, Общество]","[Политика, Президент]"
1,"Украинские войска через 10 минут после наступления нового года выпустили 15 ракет из РСЗО по Донецку, сообщили в представительстве ДНР в СЦКК","[Происшествия, Международные отношения, Региональные новости]","[Происшествия, Военные действия]"
2,❗️Пушилин сообщил о трех погибших в результате обстрела Донецка в новогоднюю ночь,"[Происшествия, Международные отношения, Региональные новости]","[Происшествия, Жертвы конфликта]"
3,"Воздушная тревога объявлена в Киеве и Киевской области, а также в Винницкой, Черкасской, Николаевской, Одесской областях и на подконтрольной Киеву территории Херсонской области","[Происшествия, Международные отношения, Региональные новости]","[Происшествия, Военные действия]"
4,"В Улан-Удэ из-за запущенного во дворе фейерверка загорелся балкон на 9-м этаже 16-этажного дома.\n\nПострадавших нет, спасатели потушили пожар на 2 квадратных метрах (видео от МЧС РФ).","[Происшествия, Региональные новости]","[Происшествия, Пожары]"
...,...,...,...
32958,"Заявка на транзит российского газа через Украину на 1 января оказалась нулевой, свидетельствуют данные оператора ГТС Украины","[Экономика, Международные отношения]","[Экономика, Энергетика]"
32959,"\n\n▪️Российские войска нанесли групповой удар по военному аэродрому и производству пороховых зарядов для ВСУ, заявили в Минобороны; \n\n▪️Российские войска нанесли поражение противнику и отразили 15 контратак, потери Киева составили до 1495 военнослужащих, добавили в ведомстве; \n\n▪️На курском направлении ВСУ за сутки потеряли более 445 военнослужащих, за всё время - до 45980 человек, говорится в отдельной сводке; \n\n▪️Эрдоган объявил, что приоритетом страны в 2025 году станет заключение справедливого мира между Россией и Украиной; \n\n▪️Почти все опрошенные ООН российские пленные на Украине заявили, что подвергались пыткам, свидетельствует доклад ООН; \n\n▪️ВСУ добивали некоторых российских солдат FPV-дронами, это военное преступление, говорится в докладе ООН; \n\n▪️Российские войска нанесли удары по технике и военным ВСУ, складу с оружием и корпусу с расчетом БПЛА в Шостке, ракетному производству в Павлограде, сообщили РИА Новости в пророссийском подполье.","[Происшествия, Политика, Международные отношения]","[Происшествия, Специальная военная операция]"
32960,"❗️Путин заслушал доклады начальника Генштаба, а также командующих группировок о ходе спецоперации и тепло поздравил с наступающим Новым годом всех военнослужащих и участников СВО, сообщил Песков","[Политика, Происшествия]","[Политика, Оборона]"


In [94]:
# df_filtered.sample(50)

In [95]:
from collections import Counter
# 'multi_labels' - это списки, их нужно преобразовать для Counter, например, в кортежи
multi_label_counts = Counter(tuple(sorted(labels)) for labels in df_filtered['multi_labels'] if labels) # sorted для учета порядка
print("Топ-10 комбинаций multi_labels:")
for labels, count in multi_label_counts.most_common(10):
    print(f"{labels}: {count}")

# Частота отдельных основных тем (если в multi_labels может быть несколько)
all_single_labels = []
for labels in df_filtered['multi_labels']:
    if labels: # Проверка, что список не пуст
        all_single_labels.extend(labels)
single_label_counts = Counter(all_single_labels)
print("\nЧастота отдельных основных тем:")
for label, count in single_label_counts.most_common():
    print(f"{label}: {count}")

Топ-10 комбинаций multi_labels:
('Международные отношения', 'Политика'): 4625
('Происшествия', 'Региональные новости'): 2887
('Международные отношения', 'Политика', 'Происшествия'): 2610
('Общество', 'Происшествия', 'Региональные новости'): 2028
('Политика', 'Происшествия', 'Региональные новости'): 1675
('Международные отношения', 'Происшествия', 'Региональные новости'): 1384
('Общество', 'Политика'): 1205
('Международные отношения', 'Политика', 'Экономика'): 1176
('Международные отношения', 'Происшествия'): 1145
('Международные отношения', 'Общество', 'Политика'): 1141

Частота отдельных основных тем:
Политика: 18637
Происшествия: 16097
Международные отношения: 15265
Региональные новости: 12043
Общество: 10886
Экономика: 4348
Технологии: 2724
Культура: 1147
Спорт: 549
Военные действия: 47
Транспорт: 30
Наука: 15
Правосудие: 9
Медиа: 8
Regиональные новости: 7
Финансы: 7
История: 6
Природа: 5
Образование: 4
Здравоохранение: 4
Криминал: 4
Безопасность: 3
Религия: 2
Экология: 2
Regionales

In [96]:
# 'hier_label' - это списки из двух элементов [тема, подтема]
hier_label_counts = Counter(tuple(labels) for labels in df_filtered['hier_label'] if labels and len(labels) == 2)
print("\nТоп-10 иерархических меток (тема, подтема):")
for labels, count in hier_label_counts.most_common(10000):
    print(f"{labels}: {count}")


Топ-10 иерархических меток (тема, подтема):
('Происшествия', 'Военные действия'): 2420
('Политика', 'Внешняя политика'): 1211
('Происшествия', 'Обстрелы'): 965
('Политика', 'Выборы'): 947
('Политика', 'Внутренняя политика'): 905
('Политика', 'Законодательство'): 599
('Международные отношения', 'Дипломатия'): 448
('Политика', 'Военные действия'): 437
('Международные отношения', 'Ближний Восток'): 417
('Происшествия', 'Атаки БПЛА'): 342
('Международные отношения', 'Военная помощь'): 297
('Общество', 'Погода'): 292
('Происшествия', 'Пожары'): 262
('Международные отношения', 'Двусторонние отношения'): 241
('Происшествия', 'Атака БПЛА'): 238
('Общество', 'Здравоохранение'): 222
('Происшествия', 'Коррупция'): 218
('Политика', 'Оборона'): 203
('Происшествия', 'Терроризм'): 199
('Происшествия', 'Преступность'): 198
('Экономика', 'Финансы'): 197
('Происшествия', 'Криминал'): 189
('Происшествия', 'ДТП'): 186
('Политика', 'Военная политика'): 181
('Политика', 'Заявления'): 176
('Политика', 'Межд

In [97]:
import pandas as pd
from IPython.display import display, HTML
from collections import Counter

# Предполагается, что df_filtered уже существует и содержит результаты от Gemini
# Если нет, загрузите или создайте его здесь.
# Пример:
# df_filtered = pd.read_pickle("processed_news_partial.pkl") # или ваш df_processed

# 0. Определяем "правильные" основные темы из вашего промпта
VALID_MAIN_LABELS = set([ # Используем set для быстрой проверки принадлежности
    "Политика", "Экономика", "Общество", "Происшествия", "Спорт",
    "Культура", "Технологии", "Международные отношения", "Региональные новости"
])

# 1. Словарь для переименования "неправильных" тем и их обработки
# Ключ - "неправильная" тема, значение - "правильная" или None для удаления
# ВАЖНО: ТЩАТЕЛЬНО ЗАПОЛНИТЕ ЭТОТ СЛОВАРЬ НА ОСНОВЕ ВАШЕГО АНАЛИЗА!
RENAME_MAP = {
    "Regиональные новости": "Региональные новости",
    "Regionales": "Региональные новости",
    "Военные действия": None, # Пример: Удаляем (или решите, куда отнести)
    "Транспорт": None,
    "Tранспорт": None,
    "Наука": "Технологии", # Пример: или Общество
    "Правосудие": "Общество", # Пример: или Политика
    "Медиа": "Общество",    # Пример: или Технологии
    "Финансы": "Экономика",
    "История": "Культура",
    "Природа": "Общество",
    "Образование": "Общество",
    "Здравоохранение": "Общество",
    "Криминал": "Происшествия",
    "Безопасность": "Общество",
    "Религия": "Общество",
    "Экология": "Общество",
    "Недвижимость": "Экономика",
    "Футбол": "Спорт",
    "Техника": "Технологии",
    "Мошенничество": "Происшествия",
    "Терроризм": "Происшествия",
    "Ближний Восток": "Международные отношения", # Можно оставить, если это частая и важная подкатегория для вас
    "Внутренняя политика": "Политика"
    # Добавьте все остальные "неправильные" темы из вашего списка!
}

def clean_and_normalize_labels_inplace(df: pd.DataFrame) -> None:
    """
    Очищает и нормализует метки в колонках 'multi_labels' и 'hier_label' DataFrame.
    Изменения производятся непосредственно в DataFrame (inplace).
    """
    if 'multi_labels' not in df.columns or 'hier_label' not in df.columns:
        print("Ошибка: DataFrame должен содержать колонки 'multi_labels' и 'hier_label'.")
        return

    # Гарантируем, что колонки содержат списки и имеют тип object
    df['multi_labels'] = df['multi_labels'].apply(lambda x: x if isinstance(x, list) else [])
    df['hier_label'] = df['hier_label'].apply(lambda x: x if (isinstance(x, list) and len(x) == 2) else [None, None])
    df['multi_labels'] = df['multi_labels'].astype(object)
    df['hier_label'] = df['hier_label'].astype(object)

    processed_rows = 0
    total_rows = len(df)

    for index, row in df.iterrows():
        multi_labels_original = list(row['multi_labels']) # Работаем с копией, чтобы не влиять на итерацию
        hier_label_original = list(row['hier_label'])

        cleaned_multi = []
        first_original_main_label = multi_labels_original[0] if multi_labels_original else None
        
        # 1. Очистка multi_labels
        for label in multi_labels_original:
            if label in VALID_MAIN_LABELS:
                if label not in cleaned_multi: # Добавляем только уникальные
                    cleaned_multi.append(label)
            elif label in RENAME_MAP:
                new_label = RENAME_MAP[label]
                if new_label and new_label not in cleaned_multi: # Если есть во что переименовать и еще не добавлено
                    cleaned_multi.append(new_label)
            # Если метки нет ни в VALID_MAIN_LABELS, ни в RENAME_MAP, она игнорируется (удаляется)
        
        df.at[index, 'multi_labels'] = cleaned_multi[:3] # Ограничиваем до 3-х и сохраняем

        # 2. Обновление hier_label на основе очищенных multi_labels
        current_first_main_after_cleaning = df.at[index, 'multi_labels'][0] if df.at[index, 'multi_labels'] else None
        
        # Если hier_label был основан на исходной первой теме multi_labels
        if hier_label_original[0] == first_original_main_label:
            # Если исходная первая тема была "неправильной" и была обработана (переименована или удалена)
            if first_original_main_label not in VALID_MAIN_LABELS and first_original_main_label in RENAME_MAP:
                mapped_value = RENAME_MAP[first_original_main_label]
                if mapped_value: # Переименована
                    df.at[index, 'hier_label'] = [mapped_value, hier_label_original[1]]
                else: # Удалена, используем новую первую из multi_labels (если есть)
                    df.at[index, 'hier_label'] = [current_first_main_after_cleaning, hier_label_original[1] if current_first_main_after_cleaning else None]
            # Если исходная первая тема была валидной, но после очистки multi_labels первая тема изменилась
            # (например, из-за удаления дубликата, который был первым)
            # или если hier_label[0] просто не совпадает с текущей первой основной темой
            elif current_first_main_after_cleaning != hier_label_original[0]:
                 df.at[index, 'hier_label'] = [current_first_main_after_cleaning, hier_label_original[1] if current_first_main_after_cleaning else None]
        # Если hier_label[0] изначально был None или не связан с первой темой multi_labels,
        # но теперь у нас есть валидная первая тема в multi_labels
        elif current_first_main_after_cleaning and hier_label_original[0] != current_first_main_after_cleaning :
            df.at[index, 'hier_label'] = [current_first_main_after_cleaning, hier_label_original[1] if current_first_main_after_cleaning else None]
        
        # Финальная проверка: если первая тема в hier_label стала None, то и подтема должна быть None
        if df.at[index, 'hier_label'][0] is None:
            df.at[index, 'hier_label'] = [None, None]
        
        processed_rows += 1
        if processed_rows % 5000 == 0:
            print(f"Очищено {processed_rows}/{total_rows} строк...")
    
    print(f"Очистка и нормализация меток завершена для {total_rows} строк.")


# --- Начало работы с DataFrame ---
# Убедимся, что колонки имеют тип object для хранения списков
df_filtered['multi_labels'] = df_filtered['multi_labels'].astype(object)
df_filtered['hier_label'] = df_filtered['hier_label'].astype(object)


print("DataFrame ДО очистки:")
display(df_filtered.head(10))

# 2. Применяем функцию очистки (изменения будут внесены в df_filtered)
clean_and_normalize_labels_inplace(df_filtered)

print("\nDataFrame ПОСЛЕ очистки:")
display(df_filtered.head(10))

# 3. Показываем строки, где первая тема multi_labels не соответствует первой теме hier_label
#    (исключая случаи, когда одна из них или обе пусты/None)
mismatched_rows_indices = []
for index, row in df_filtered.iterrows():
    ml = row['multi_labels']
    hl = row['hier_label']

    first_ml = ml[0] if ml else None
    first_hl = hl[0] if hl and hl[0] is not None else None # Убедимся, что hl[0] не None

    if first_ml is not None and first_hl is not None and first_ml != first_hl:
        mismatched_rows_indices.append(index)
    elif (first_ml is not None and first_hl is None and ml): # есть multi, но hier[0] пуст
        mismatched_rows_indices.append(index)

# 4. Для проверки, какие темы остались в multi_labels
print("\nЧастота отдельных основных тем ПОСЛЕ ОЧИСТКИ:")
all_single_labels_cleaned = []
for labels_list in df_filtered['multi_labels']:
    if labels_list: # Проверка, что список не пуст
        all_single_labels_cleaned.extend(labels_list)
single_label_counts_cleaned = Counter(all_single_labels_cleaned)
for label, count in single_label_counts_cleaned.most_common():
    print(f"{label}: {count}")

DataFrame ДО очистки:


,text,multi_labels,hier_label
id,,,
0,Новогоднее обращение Владимира Путина,"[Политика, Общество]","[Политика, Президент]"
1,"Украинские войска через 10 минут после наступления нового года выпустили 15 ракет из РСЗО по Донецку, сообщили в представительстве ДНР в СЦКК","[Происшествия, Международные отношения, Региональные новости]","[Происшествия, Военные действия]"
2,❗️Пушилин сообщил о трех погибших в результате обстрела Донецка в новогоднюю ночь,"[Происшествия, Международные отношения, Региональные новости]","[Происшествия, Жертвы конфликта]"
3,"Воздушная тревога объявлена в Киеве и Киевской области, а также в Винницкой, Черкасской, Николаевской, Одесской областях и на подконтрольной Киеву территории Херсонской области","[Происшествия, Международные отношения, Региональные новости]","[Происшествия, Военные действия]"
4,"В Улан-Удэ из-за запущенного во дворе фейерверка загорелся балкон на 9-м этаже 16-этажного дома.\n\nПострадавших нет, спасатели потушили пожар на 2 квадратных метрах (видео от МЧС РФ).","[Происшествия, Региональные новости]","[Происшествия, Пожары]"
5,"Ким Чен Ын на новогодней встрече с командованием армии заявил, что если США и Южная Корея выберут путь конфронтации и вызовут конфликт,","[Международные отношения, Политика]","[Международные отношения, Военная политика]"
6,"Вулкан Ключевской на Камчатке в ходе нового извержения выбросил столб пепла до 6 км, шлейф пепла протянулся более чем на 100 км.\n\nПредыдущее извержение длилось с 23 июня по 1 ноября и завершилось крупным выбросом пепла на высоту до 14 км. Частицы пепла тогда выпали в Скандинавии.\n\nФото Камчатской группы реагирования на вулканические извержения Института вулканологии и сейсмологии ДВО РАН.","[Происшествия, Региональные новости]","[Происшествия, Стихийные бедствия]"
7,"Объект припортовой инфраструктуры поврежден в Одессе из-за взрывов, там произошел пожар, сообщили украинские военные","[Происшествия, Международные отношения, Региональные новости]","[Происшествия, Военные действия]"
8,"Музей украинскому националисту Шухевичу уничтожен во Львове из-за взрывов, сообщил мэр города","[Происшествия, Международные отношения, Культура]","[Происшествия, Военные действия]"


Очищено 5000/32902 строк...
Очищено 10000/32902 строк...
Очищено 15000/32902 строк...
Очищено 20000/32902 строк...
Очищено 25000/32902 строк...
Очищено 30000/32902 строк...
Очистка и нормализация меток завершена для 32902 строк.

DataFrame ПОСЛЕ очистки:


,text,multi_labels,hier_label
id,,,
0,Новогоднее обращение Владимира Путина,"[Политика, Общество]","[Политика, Президент]"
1,"Украинские войска через 10 минут после наступления нового года выпустили 15 ракет из РСЗО по Донецку, сообщили в представительстве ДНР в СЦКК","[Происшествия, Международные отношения, Региональные новости]","[Происшествия, Военные действия]"
2,❗️Пушилин сообщил о трех погибших в результате обстрела Донецка в новогоднюю ночь,"[Происшествия, Международные отношения, Региональные новости]","[Происшествия, Жертвы конфликта]"
3,"Воздушная тревога объявлена в Киеве и Киевской области, а также в Винницкой, Черкасской, Николаевской, Одесской областях и на подконтрольной Киеву территории Херсонской области","[Происшествия, Международные отношения, Региональные новости]","[Происшествия, Военные действия]"
4,"В Улан-Удэ из-за запущенного во дворе фейерверка загорелся балкон на 9-м этаже 16-этажного дома.\n\nПострадавших нет, спасатели потушили пожар на 2 квадратных метрах (видео от МЧС РФ).","[Происшествия, Региональные новости]","[Происшествия, Пожары]"
5,"Ким Чен Ын на новогодней встрече с командованием армии заявил, что если США и Южная Корея выберут путь конфронтации и вызовут конфликт,","[Международные отношения, Политика]","[Международные отношения, Военная политика]"
6,"Вулкан Ключевской на Камчатке в ходе нового извержения выбросил столб пепла до 6 км, шлейф пепла протянулся более чем на 100 км.\n\nПредыдущее извержение длилось с 23 июня по 1 ноября и завершилось крупным выбросом пепла на высоту до 14 км. Частицы пепла тогда выпали в Скандинавии.\n\nФото Камчатской группы реагирования на вулканические извержения Института вулканологии и сейсмологии ДВО РАН.","[Происшествия, Региональные новости]","[Происшествия, Стихийные бедствия]"
7,"Объект припортовой инфраструктуры поврежден в Одессе из-за взрывов, там произошел пожар, сообщили украинские военные","[Происшествия, Международные отношения, Региональные новости]","[Происшествия, Военные действия]"
8,"Музей украинскому националисту Шухевичу уничтожен во Львове из-за взрывов, сообщил мэр города","[Происшествия, Международные отношения, Культура]","[Происшествия, Военные действия]"



Частота отдельных основных тем ПОСЛЕ ОЧИСТКИ:
Политика: 18635
Происшествия: 16096
Международные отношения: 15265
Региональные новости: 12048
Общество: 10893
Экономика: 4347
Технологии: 2728
Культура: 1152
Спорт: 549


In [17]:
MIN_HIER_LABEL_COUNT = 50 # Установите ваш порог

# Сначала посчитаем частоту всех иерархических меток
hier_label_tuples = [tuple(labels) for labels in df_filtered['hier_label'] if labels and len(labels) == 2 and all(labels)]
hier_label_counts = Counter(hier_label_tuples)

def consolidate_rare_hier_labels(row):
    hier_label = row['hier_label']
    if hier_label and len(hier_label) == 2 and all(hier_label): # Проверяем, что метка валидна
        current_tuple = tuple(hier_label)
        if hier_label_counts[current_tuple] < MIN_HIER_LABEL_COUNT:
            # Заменяем подтему на "Другое", оставляя основную тему
            return [hier_label[0], "Другое"] 
    return hier_label # Возвращаем как есть, если метка частая или невалидная/пустая

# Применяем только к тем строкам, где hier_label не пустой и корректный
# Создадим новую колонку для безопасности или обновим существующую
df_filtered['hier_label'] = df_filtered.apply(consolidate_rare_hier_labels, axis=1)

# Посмотрим на новые частоты
hier_label_consolidated_tuples = [tuple(labels) for labels in df_filtered['hier_label'] if labels and len(labels) == 2 and all(labels)]
hier_label_consolidated_counts = Counter(hier_label_consolidated_tuples)
print("\nТоп иерархических меток ПОСЛЕ консолидации редких менее 30:")
for labels, count in hier_label_consolidated_counts.most_common(120):
    print(f"{labels}: {count}")

# Посмотрим, сколько теперь меток "Другое" для каждой основной темы
other_counts = Counter(item[0] for item in hier_label_consolidated_counts if item[1] == "Другое")
print("\nКоличество подтем 'Другое' для каждой основной темы:")
for main_theme, count in other_counts.items():
     # Ищем общее количество для этой основной темы
    total_for_main_theme = sum(v for k, v in hier_label_consolidated_counts.items() if k[0] == main_theme)
    print(f"{main_theme}: {hier_label_consolidated_counts[(main_theme, 'Другое')]} (из {total_for_main_theme} всего для {main_theme})")

# Если хотите обновить исходную колонку:
# df_filtered['hier_label'] = df_filtered['hier_label_consolidated']
# df_filtered.drop(columns=['hier_label_consolidated'], inplace=True)


Топ иерархических меток ПОСЛЕ консолидации редких менее 30:
('Происшествия', 'Другое'): 3007
('Политика', 'Другое'): 2691
('Происшествия', 'Военные действия'): 2419
('Международные отношения', 'Другое'): 2252
('Общество', 'Другое'): 1860
('Экономика', 'Другое'): 1269
('Политика', 'Внешняя политика'): 1214
('Происшествия', 'Обстрелы'): 1075
('Политика', 'Выборы'): 947
('Политика', 'Внутренняя политика'): 905
('Региональные новости', 'Другое'): 679
('Политика', 'Законодательство'): 599
('Международные отношения', 'Дипломатия'): 447
('Политика', 'Военные действия'): 438
('Происшествия', 'Пожары'): 436
('Международные отношения', 'Ближний Восток'): 414
('Технологии', 'Другое'): 401
('Происшествия', 'Атаки БПЛА'): 342
('Культура', 'Другое'): 340
('Международные отношения', 'Военная помощь'): 294
('Общество', 'Погода'): 292
('Спорт', 'Другое'): 261
('Происшествия', 'Преступность'): 250
('Происшествия', 'Атака БПЛА'): 238
('Международные отношения', 'Двусторонние отношения'): 237
('Общество'

In [27]:
df_filtered.to_csv('2024_y.csv', index=False)

In [26]:
import pandas as pd
from collections import Counter
from IPython.display import display, HTML
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import ast # Для преобразования строк в списки
import json

df_filtered = pd.read_csv('2024_y.csv')

# --- ПРЕОБРАЗОВАНИЕ СТРОК В СПИСКИ ---
def safe_literal_eval(val):
    if isinstance(val, str): # Обрабатываем только строки
        try:
            # Попробуем сначала как JSON, так как Gemini возвращает JSON-совместимые списки
            # Это лучше справится с кавычками внутри строк, если они есть
            res = json.loads(val.replace("'", "\"")) # Заменяем одинарные кавычки на двойные для json.loads
            if isinstance(res, list) and len(res) == 2 and all(isinstance(i, str) for i in res):
                 return res
            # Если не получилось как JSON или структура не та, пробуем ast.literal_eval
            # или возвращаем формат по умолчанию для ошибок
        except (json.JSONDecodeError, TypeError):
            try:
                res_ast = ast.literal_eval(val)
                if isinstance(res_ast, list) and len(res_ast) == 2 and all(isinstance(i, str) for i in res_ast):
                    return res_ast
            except (ValueError, SyntaxError, TypeError):
                pass # Оставляем как есть или возвращаем пустой/дефолтный список, если не удалось
    elif isinstance(val, list) and len(val) == 2 and all(isinstance(i, str) for i in val): # Если уже правильный список
        return val
    return [None, None] # Возвращаем структуру по умолчанию, если не удалось преобразовать или не список

if 'hier_label' in df_filtered.columns:
    print("Преобразование колонки 'hier_label' из строк в списки...")
    df_filtered['hier_label'] = df_filtered['hier_label'].apply(safe_literal_eval)
    # Проверим тип после преобразования
    print(f"Тип данных в колонке 'hier_label' после преобразования: {type(df_filtered['hier_label'].iloc[0]) if not df_filtered.empty else 'DataFrame пуст'}")
    # Убедимся, что это тип object для хранения списков
    df_filtered['hier_label'] = df_filtered['hier_label'].astype(object)
else:
    print("ПРЕДУПРЕЖДЕНИЕ: Колонка 'hier_label' отсутствует в загруженном CSV.")
    # Создадим пустую колонку, чтобы остальной код не упал, но объединения не будет
    df_filtered['hier_label'] = pd.Series([[] for _ in range(len(df_filtered))], index=df_filtered.index, dtype=object)

# 0. Параметры
MODEL_NAME_EMBEDDINGS = 'cointegrated/rubert-tiny2' # Модель для получения эмбеддингов
SIMILARITY_THRESHOLD = 0.9 # Порог косинусного сходства для объединения (подбирается экспериментально, 0.8-0.95)
MIN_SUBTHEME_COUNT_FOR_MERGE_TARGET = 10 # Не объединять В подтему, если она сама по себе очень редкая (можно настроить)

print(f"Загрузка модели для эмбеддингов: {MODEL_NAME_EMBEDDINGS}...")
try:
    embedder = SentenceTransformer(MODEL_NAME_EMBEDDINGS)
    print("Модель загружена.")
except Exception as e:
    print(f"Ошибка загрузки модели эмбеддингов: {e}")
    print("Дальнейшее объединение подтем по семантической близости невозможно без модели.")
    # Можно здесь остановить выполнение или пропустить этот шаг
    exit()


def merge_similar_subthemes_in_dataframe(df, hier_label_col='hier_label'):
    """
    Объединяет семантически похожие подтемы внутри каждой основной темы в DataFrame.
    Изменения производятся в новой колонке 'hier_label_merged'.
    """
    if hier_label_col not in df.columns:
        print(f"Ошибка: Колонка '{hier_label_col}' отсутствует в DataFrame.")
        return df

    df_copy = df.copy()
    # Создаем новую колонку, чтобы не изменять исходную во время итераций по ней же
    df_copy['hier_label_merged'] = df_copy[hier_label_col].copy().astype(object)

    # 1. Собираем все уникальные пары (основная_тема, подтема) и их частоты
    valid_hier_labels = [tuple(labels) for labels in df_copy[hier_label_col]
                         if isinstance(labels, list) and len(labels) == 2 and all(isinstance(l, str) for l in labels)]
    hier_label_counts = Counter(valid_hier_labels)

    # 2. Группируем подтемы по основным темам
    main_theme_to_subthemes = {}
    for (main_theme, sub_theme), count in hier_label_counts.items():
        if main_theme not in main_theme_to_subthemes:
            main_theme_to_subthemes[main_theme] = []
        main_theme_to_subthemes[main_theme].append({'subtheme': sub_theme, 'count': count, 'original_tuple': (main_theme, sub_theme)})

    print(f"\nНачинаем объединение похожих подтем. Порог сходства: {SIMILARITY_THRESHOLD}")

    # 3. Для каждой основной темы объединяем похожие подтемы
    final_rename_map_for_subthemes = {} # (main, old_sub) -> (main, new_sub)

    for main_theme, subtheme_dicts in main_theme_to_subthemes.items():
        if len(subtheme_dicts) < 2: # Нечего объединять, если подтема одна или нет
            continue

        # Исключаем "Другое" из процесса объединения (не хотим объединять что-то В "Другое" на этом этапе,
        # и не хотим, чтобы "Другое" стало целью для других)
        subthemes_to_process = [st for st in subtheme_dicts if st['subtheme'] != "Другое"]
        
        if len(subthemes_to_process) < 2:
            continue

        print(f"\nОбработка основной темы: {main_theme} (уникальных подтем, кроме 'Другое': {len(subthemes_to_process)})")

        # Сортируем подтемы по убыванию частоты, чтобы более частые были приоритетными целями для слияния
        subthemes_to_process.sort(key=lambda x: x['count'], reverse=True)
        
        subtheme_names = [st['subtheme'] for st in subthemes_to_process]
        
        if not subtheme_names: continue

        embeddings = embedder.encode(subtheme_names, show_progress_bar=False)
        cosine_matrix = cosine_similarity(embeddings)
        
        merged_into_this_iteration = set() # Чтобы не объединять одну и ту же подтему несколько раз за итерацию
        
        # Итеративно объединяем. Можно сделать несколько проходов, но начнем с одного.
        # Мы хотим объединять более редкие в более частые
        for i in range(len(subthemes_to_process)):
            if subtheme_names[i] in merged_into_this_iteration:
                continue # Эту подтему уже во что-то объединили

            current_subtheme_info = subthemes_to_process[i]
            
            for j in range(len(subthemes_to_process)): # Можно было бы начинать с i+1, но т.к. сортировка по частоте, ищем цель среди всех
                if i == j or subtheme_names[j] in merged_into_this_iteration:
                    continue

                target_subtheme_info = subthemes_to_process[j]

                # Условие: объединяем i-ю в j-ю, если j-я чаще или равна по частоте И j-я не реже порога
                # ИЛИ если i-я значительно реже j-й, даже если j-я ниже порога (но не "Другое")
                # И сходство выше порога.
                # Более простая логика: объединяем менее частую в более частую, если они похожи.
                # Так как мы отсортировали, subthemes_to_process[i] будет более частой или равной subthemes_to_process[j], если j > i.
                # Поэтому будем пытаться объединить subthemes_to_process[j] (более редкую) в subthemes_to_process[i] (более частую)
                # если i < j (т.е. i-я более частая)
                
                # Перебираем пары: (более частая, менее частая)
                # Пусть target = более частая, current = менее частая
                # Мы отсортировали, так что subthemes_to_process[idx_target] будет более частой, если idx_target < idx_current
                
                # Пройдемся по всем парам и найдем кандидатов на слияние
                # i - кандидат на то, чтобы стать "целью" слияния (более частый)
                # j - кандидат на то, чтобы быть слитым в i (менее частый)
                
                # Мы уже в цикле по i (потенциальная цель, отсортировано по убыванию частоты)
                # Теперь ищем j > i (менее частые), которые можно слить в i
                # Но для простоты перебора всех пар, сохраним текущую логику,
                # а решение о том, кто в кого сливается, примем на основе частот

                similarity = cosine_matrix[i, j]

                if similarity >= SIMILARITY_THRESHOLD:
                    # Определяем, кто в кого будет сливаться: менее частый в более частого
                    sub_i_info = subthemes_to_process[i]
                    sub_j_info = subthemes_to_process[j]

                    # Пропускаем, если одна из подтем уже была целью слияния на этой итерации
                    if sub_i_info['subtheme'] in merged_into_this_iteration or \
                       sub_j_info['subtheme'] in merged_into_this_iteration:
                        continue
                    
                    # Цель слияния (target) - более частая, источник (source) - менее частая
                    if sub_i_info['count'] >= sub_j_info['count']:
                        target_info, source_info = sub_i_info, sub_j_info
                    else:
                        target_info, source_info = sub_j_info, sub_i_info
                    
                    # Не объединяем, если целевая подтема сама по себе слишком редкая (кроме случая, когда источник еще реже)
                    if target_info['count'] < MIN_SUBTHEME_COUNT_FOR_MERGE_TARGET and source_info['count'] >= target_info['count']:
                        # print(f"    Пропуск слияния: '{source_info['subtheme']}' ({source_info['count']}) -> '{target_info['subtheme']}' ({target_info['count']}) (цель слишком редкая)")
                        continue

                    print(f"    Объединяем: '{source_info['subtheme']}' ({source_info['count']}) -> '{target_info['subtheme']}' ({target_info['count']}) (сходство: {similarity:.3f})")
                    final_rename_map_for_subthemes[source_info['original_tuple']] = target_info['original_tuple']
                    merged_into_this_iteration.add(source_info['subtheme']) # Добавляем имя подтемы, которую объединили

    # 4. Применяем созданный final_rename_map_for_subthemes к DataFrame
    if final_rename_map_for_subthemes:
        print(f"\nПрименение {len(final_rename_map_for_subthemes)} правил объединения подтем к DataFrame...")
        
        def apply_subtheme_merging(hier_label_list):
            if isinstance(hier_label_list, list) and len(hier_label_list) == 2 and all(isinstance(l, str) for l in hier_label_list):
                current_tuple = tuple(hier_label_list)
                # Идем по цепочке переименований, если есть
                while current_tuple in final_rename_map_for_subthemes:
                    current_tuple = final_rename_map_for_subthemes[current_tuple]
                return list(current_tuple)
            return hier_label_list

        # Применяем к КОПИИ КОЛОНКИ, чтобы использовать .at для присвоения
        df_copy['hier_label_merged'] = df_copy['hier_label_merged'].apply(apply_subtheme_merging)
        print("Применение объединения подтем завершено.")
    else:
        print("Не найдено подтем для семантического объединения с текущим порогом.")
        
    return df_copy


# --- Начало работы с DataFrame ---
# Убедитесь, что df_filtered существует и содержит колонку 'hier_label'
# Если ее нет, создайте ее на основе 'hier_label' с объединением редких в "Другое"
# (код для этого был в предыдущем ответе)

if 'hier_label' not in df_filtered.columns:
    print("Колонка 'hier_label' не найдена. Пожалуйста, создайте ее сначала (объединив редкие подтемы в 'Другое').")
    # df_filtered['hier_label'] = df_filtered['hier_label'].copy() # Заглушка, если нужно запустить
else:
    print("DataFrame ДО семантического объединения подтем (на основе 'hier_label'):")
    hier_counts_before_semantic = Counter(tuple(l) for l in df_filtered['hier_label'] if isinstance(l, list) and len(l)==2 and all(l))
    # display(pd.Series(hier_counts_before_semantic).sort_values(ascending=False).head(30))
    print("Топ-30 иерархических меток ДО семантического объединения:")
    for labels, count in hier_counts_before_semantic.most_common(30):
            print(f"{labels}: {count}")


    # Применяем семантическое объединение
    df_filtered_after_semantic_merge = merge_similar_subthemes_in_dataframe(df_filtered, hier_label_col='hier_label')

    print("\nDataFrame ПОСЛЕ семантического объединения подтем:")
    # display(df_filtered_after_semantic_merge[['hier_label', 'hier_label_merged']].head(20))
    
    hier_counts_after_semantic = Counter(tuple(l) for l in df_filtered_after_semantic_merge['hier_label_merged'] if isinstance(l, list) and len(l)==2 and all(l))
    # display(pd.Series(hier_counts_after_semantic).sort_values(ascending=False).head(30))
    print("\nТоп-30 иерархических меток ПОСЛЕ семантического объединения:")
    for labels, count in hier_counts_after_semantic.most_common(30):
            print(f"{labels}: {count}")
    
    # Если хотите обновить исходную колонку в df_filtered:
    df_filtered['hier_label'] = df_filtered_after_semantic_merge['hier_label_merged']
    df_filtered.drop(columns=['hier_label_merged'], inplace=True, errors='ignore')

    # Проверка, сколько уникальных осталось
    num_unique_before = len(hier_counts_before_semantic)
    num_unique_after = len(hier_counts_after_semantic)
    print(f"\nКоличество уникальных иерархических меток: {num_unique_before} -> {num_unique_after}")

    # Показать примеры, где произошло объединение
    # Создадим столбец с исходным значением для сравнения, если 'hier_label_merged' отличается
    df_filtered_after_semantic_merge['temp_hier_consolidated_tuple'] = df_filtered_after_semantic_merge['hier_label'].apply(lambda x: tuple(x) if isinstance(x,list) and len(x)==2 else None)
    df_filtered_after_semantic_merge['temp_hier_merged_tuple'] = df_filtered_after_semantic_merge['hier_label_merged'].apply(lambda x: tuple(x) if isinstance(x,list) and len(x)==2 else None)
    
    df_changed_semantic = df_filtered_after_semantic_merge[
        df_filtered_after_semantic_merge['temp_hier_consolidated_tuple'].notna() &
        df_filtered_after_semantic_merge['temp_hier_merged_tuple'].notna() &
        (df_filtered_after_semantic_merge['temp_hier_consolidated_tuple'] != df_filtered_after_semantic_merge['temp_hier_merged_tuple'])
    ]
    if not df_changed_semantic.empty:
        print(f"\nПримеры строк, где подтемы были семантически объединены ({len(df_changed_semantic)} строк):")
        display(df_changed_semantic[['hier_label', 'hier_label_merged']].head(10))
    
    df_filtered_after_semantic_merge.drop(columns=['temp_hier_consolidated_tuple', 'temp_hier_merged_tuple'], inplace=True, errors='ignore')

Преобразование колонки 'hier_label' из строк в списки...
Тип данных в колонке 'hier_label' после преобразования: <class 'list'>
Загрузка модели для эмбеддингов: cointegrated/rubert-tiny2...
Модель загружена.
DataFrame ДО семантического объединения подтем (на основе 'hier_label'):
Топ-30 иерархических меток ДО семантического объединения:
('Происшествия', 'Другое'): 3007
('Политика', 'Другое'): 2691
('Происшествия', 'Военные действия'): 2419
('Международные отношения', 'Другое'): 2252
('Общество', 'Другое'): 1860
('Экономика', 'Другое'): 1269
('Политика', 'Внешняя политика'): 1214
('Происшествия', 'Обстрелы'): 1075
('Политика', 'Выборы'): 947
('Политика', 'Внутренняя политика'): 905
('Региональные новости', 'Другое'): 679
('Политика', 'Законодательство'): 599
('Международные отношения', 'Дипломатия'): 447
('Политика', 'Военные действия'): 438
('Происшествия', 'Пожары'): 436
('Международные отношения', 'Ближний Восток'): 414
('Технологии', 'Другое'): 401
('Происшествия', 'Атаки БПЛА'): 34

/Users/maksimlyara/miniconda3/envs/project_dl_nlp/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/maksimlyara/miniconda3/envs/project_dl_nlp/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/maksimlyara/miniconda3/envs/project_dl_nlp/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/Users/maksimlyara/miniconda3/envs/project_dl_nlp/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/maksimlyara/miniconda3/envs/project_dl_nlp/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/maksimlyara/miniconda3/envs/project_dl_nlp/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid 

,hier_label,hier_label_merged
51,"[Происшествия, Нападения]","[Происшествия, Коррупция]"
59,"[Происшествия, Нападения]","[Происшествия, Коррупция]"
60,"[Происшествия, Мошенничество]","[Происшествия, Преступность]"
87,"[Происшествия, Взрывы]","[Происшествия, Обстрелы]"
114,"[Происшествия, Транспортные происшествия]","[Происшествия, Транспорт]"
168,"[Политика, Миграционная политика]","[Политика, Внешняя политика]"
176,"[Происшествия, Транспортные происшествия]","[Происшествия, Транспорт]"
181,"[Происшествия, Транспортные происшествия]","[Происшествия, Транспорт]"
207,"[Происшествия, Взрывы]","[Происшествия, Обстрелы]"
240,"[Происшествия, Транспортные происшествия]","[Происшествия, Транспорт]"


In [20]:
df_filtered

,text,multi_labels,hier_label
0,Новогоднее обращение Владимира Путина,"['Политика', 'Общество']","[Политика, Другое]"
1,Украинские войска через 10 минут после наступл...,"['Происшествия', 'Международные отношения', 'Р...","[Происшествия, Военные действия]"
2,❗️Пушилин сообщил о трех погибших в результате...,"['Происшествия', 'Международные отношения', 'Р...","[Происшествия, Другое]"
3,Воздушная тревога объявлена в Киеве и Киевской...,"['Происшествия', 'Международные отношения', 'Р...","[Происшествия, Военные действия]"
4,В Улан-Удэ из-за запущенного во дворе фейервер...,"['Происшествия', 'Региональные новости']","[Происшествия, Пожары]"
...,...,...,...
32897,Заявка на транзит российского газа через Украи...,"['Экономика', 'Международные отношения']","[Экономика, Энергетика]"
32898,\n\n▪️Российские войска нанесли групповой удар...,"['Происшествия', 'Политика', 'Международные от...","[Происшествия, Другое]"
32899,"❗️Путин заслушал доклады начальника Генштаба, ...","['Политика', 'Происшествия']","[Политика, Оборона]"
32900,Повторные выбросы нефтепродуктов зафиксированы...,"['Происшествия', 'Региональные новости', 'Обще...","[Происшествия, Другое]"
